# Import Library


In [6]:
import shutil
import requests
import time
import os
import json
import datetime
import tqdm

# Scraping

- Scraping from MyAnimeList one by one are slow
- Instead we scrape use database from jikan all thanks to https://github.com/andreu-vall/myanimelist-api-scraping/


In [8]:
def merge_files(tmp_directory, database):

    data = []
    for file_name in os.listdir(tmp_directory):
        file_path = os.path.join(tmp_directory, file_name)
        with open(file_path, 'r') as f:
            file = json.load(f)
        data.extend(file)
    
    with open(f'../dataset/raw/{database}_jikan.json', 'w') as f:
        json.dump(data, f, indent=4)
    
    shutil.rmtree(tmp_directory)

In [9]:
api_url = 'https://api.jikan.moe/v4'

def scrape_page(endpoint, page, file_path):
    response = requests.get(api_url + endpoint + f'?page={page}')
    response.raise_for_status()
    data = response.json()
    with open(file_path, 'w') as f:
        json.dump(data['data'], f, indent=4)

wait = 1.2

def scrape_jikan_db(database):
    tmp_directory = f'../dataset/raw/tmp_{database}_jikan'
    os.makedirs(tmp_directory)
    
    last_page = requests.get(api_url + '/' + database).json()['pagination']['last_visible_page']
    length = len(str(last_page))

    start = datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")
    print('Scraped at:', start)
    
    for page in tqdm.trange(1, last_page + 1):
        start = time.perf_counter()
        scrape_page('/' + database, page, tmp_directory + f'/page{str(page).zfill(length)}.json')
        end = time.perf_counter()
        time.sleep(max(0, start + wait - end))
    
    merge_files(tmp_directory, database)

In [11]:
scrape_jikan_db('anime')

Scraped at: 2025-01-15 14.34.02


100%|██████████| 1119/1119 [24:40<00:00,  1.32s/it]
